In [34]:
import os, glob
import pandas as pd

gzs = './data/*/eda//*.csv'
paths = glob.glob(gzs)

In [35]:
# generate word level
ds_names = []
dfs = []
for p in paths:
    if 'infor_word' in p:
        ds_names.append(p.split('\\')[1])
        dfs.append(pd.read_csv(p))

df = pd.concat(dfs)
df['dataset'] = ds_names

df = df[['dataset', 'avg_fa', 'min_fa', 'max_fa', '92%_fa', 'all_fa', 'unique_fa', 'avg_en',
       'min_en', 'max_en', '92%_en', 'all_en', 'unique_en']]

df.to_csv('./word_level_info.csv', index=False)

In [36]:
# generate charachter level
ds_names = []
dfs = []
for p in paths:
    if 'infor_char' in p:
        ds_names.append(p.split('\\')[1])
        dfs.append(pd.read_csv(p))

df = pd.concat(dfs)
df['dataset'] = ds_names
df = df[['dataset', 'avgc_fa', 'minc_fa', 'maxc_fa', 'avgc_en', 'minc_en', 'maxc_en']]
df.to_csv('./char_level_info.csv', index=False)

